In [1]:
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from scapy.all import *

Using TensorFlow backend.


In [2]:
import re
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
def preprocess_data(file):
    Payloads = []
    with PcapReader(file) as pr:
        for packet in pr:        
            Payloads.append(str(packet[TCP].payload))
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(Payloads)


    Data = []
    for payload in Payloads:
        payload = re.sub(r'[^a-zA-Z0-9.\s]', ' ', payload)
        payload = [payload]
        sequences = tokenizer.texts_to_sequences(payload)
        Data.append(preprocessing.sequence.pad_sequences(sequences, maxlen = 200)[0])
    return Data

In [4]:
Data = preprocess_data('/home/divyaansh/HTTP_For_RangeGram/Normal.pcap')

In [5]:
X_train, X_test = train_test_split(Data, test_size=0.2)

In [6]:
import numpy as np

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

In [7]:
from sklearn import svm
svm_model = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1, verbose = 3)
svm_model.fit(X_train)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=3)

In [8]:
from joblib import dump, load
dump(svm_model, 'svm_model.joblib')

['svm_model.joblib']

In [9]:
attack_data = preprocess_data('/home/arun/HTTP_For_RangeGram/ShellCodeAttack.pcap')
data = np.asarray(attack_data)

In [10]:
pred = svm_model.predict(data)

In [11]:
pred

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1])

In [12]:
c = 0
for i in pred :
    if i == -1:
        c+=1
print("Accuracy on Shell Code Attack:",c/len(pred))

Accuracy on Shell Code Attack: 1.0


In [13]:
attack_data2  = preprocess_data('/home/arun/HTTP_For_RangeGram/GenericAttack.pcap')
data = np.asarray(attack_data2)

In [14]:
pred2 = svm_model.predict(data)

In [15]:
pred2

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1])

In [16]:
c = 0
for i in pred2 :
    if i == -1:
        c+=1
print("Accuracy on generic attack:",c/len(pred2))

Accuracy on generic attack: 1.0


In [17]:
pred3 = svm_model.predict(X_test)

In [18]:
c = 0
for i in pred3 :
    if i == 1:
        c+=1
print("Accuracy on normal packets: ", c/len(pred3))

Accuracy on normal packets:  0.1484182610342868


In [19]:
attack_data2  = preprocess_data('/home/arun/HTTP_For_RangeGram/MorphedShellCodeAttack.pcap')
data = np.asarray(attack_data2)

In [20]:
pred4 = svm_model.predict(data)

In [21]:
c = 0
for i in pred4 :
    if i == -1:
        c+=1
print("Accuracy on morphed shell code attack:",c/len(pred4))

Accuracy on morphed shell code attack: 1.0
